In [ ]:
from scipy.io import loadmat
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Flatten
from keras.optimizers import Adam

In [ ]:
def load_data_from_mat(file_path, key):
    data = loadmat(file_path)
    return data[key]


In [ ]:
# Define paths and keys for each class
file_info_dict = {
    'A': [('/content/drive/MyDrive/emdvmd/VMDF.mat', 'IMFvmd1F')],
    'B': [('/content/drive/MyDrive/emdvmd/VMDN.mat', 'IMFvmd1N')],
    'C': [('/content/drive/MyDrive/emdvmd/VMDO.mat', 'IMFvmd1O')],
    'D': [('/content/drive/MyDrive/emdvmd/VMDS.mat', 'IMFvmd1S')],
    'E': [('/content/drive/MyDrive/emdvmd/VMDZ.mat', 'IMFvmd1Z')]
}

In [ ]:
def prepare_data_for_classification(classes):
    X, y = [], []
    for label, class_id in enumerate(classes):
        file_info = file_info_dict[class_id]  # Get file paths and keys for each class
        for file_path, key in file_info:
            data = load_data_from_mat(file_path, key)
            X.append(data)
            y.append(np.array([label] * data.shape[0]))  # Assign the same label to all samples
    X = np.concatenate(X)
    y = np.concatenate(y)
    return X, y


In [ ]:
def create_lstm_model(input_shape, lstm_units1, lstm_units2, dense_units, output_units, dropout_rate):
    model = Sequential()
    model.add(LSTM(units=lstm_units1, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(rate=dropout_rate))
    model.add(LSTM(units=lstm_units2))
    model.add(Dropout(rate=dropout_rate))
    model.add(Dense(units=dense_units, activation='relu'))
    model.add(Dense(units=output_units, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def compute_metrics(y_true, y_pred):
    y_true = np.argmax(y_true, axis=1)
    y_pred = np.argmax(y_pred, axis=1)
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred, average='macro')
    specificity = recall_score(y_true, y_pred, average='macro', labels=[0, 1])
    f1 = f1_score(y_true, y_pred, average='macro')
    return accuracy, sensitivity, specificity, f1

In [ ]:
def train_and_evaluate_lstm_model(X_train, y_train, X_test, y_test, lstm_units1, lstm_units2, dense_units, dropout_rate, batch_size, epochs):
    model = create_lstm_model((X_train.shape[1], 1), lstm_units1, lstm_units2, dense_units, y_train.shape[1], dropout_rate)
    history = model.fit(X_train, y_train, validation_split=0.2, epochs=epochs, batch_size=batch_size, verbose=1)
    y_pred = model.predict(X_test)
    score = model.evaluate(X_test, y_test, verbose=1)
    accuracy, sensitivity, specificity, f1 = compute_metrics(y_test, y_pred)
    print(f'Test loss: {score[0]}')
    print(f'Test accuracy: {accuracy}')
    print(f'Sensitivity (Recall): {sensitivity}')
    print(f'Specificity: {specificity}')
    print(f'F1 Score: {f1}')

Classification A-B vs. C-D-E

In [ ]:
# Load and preprocess data for classes A-B vs. C-D-E
classes_ab = ['A', 'B']
classes_cde = ['C', 'D', 'E']

X_ab, y_ab = prepare_data_for_classification(classes_ab)
X_cde, y_cde = prepare_data_for_classification(classes_cde)

# Merge and create new labels
X = np.concatenate([X_ab, X_cde])
y = np.concatenate([np.zeros(len(y_ab)), np.ones(len(y_cde))])  # 0 for A-B, 1 for C-D-E

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Encode labels
y = y.astype(int)
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the LSTM model
train_and_evaluate_lstm_model(X_train, y_train, X_test, y_test, lstm_units1=128, lstm_units2=64, dense_units=32, dropout_rate=0.3, batch_size=32, epochs=20)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 104s 232ms/step - accuracy: 0.7146 - loss: 0.5397 - val_accuracy: 0.9002 - val_loss: 0.2517
Epoch 2/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 136s 218ms/step - accuracy: 0.8833 - loss: 0.2836 - val_accuracy: 0.6056 - val_loss: 0.6712
Epoch 3/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 138s 209ms/step - accuracy: 0.5969 - loss: 0.6754 - val_accuracy: 0.6056 - val_loss: 0.6712
Epoch 4/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 145s 217ms/step - accuracy: 0.5976 - loss: 0.6733 - val_accuracy: 0.6056 - val_loss: 0.6718
Epoch 5/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 139s 209ms/step - accuracy: 0.5993 - loss: 0.6698 - val_accuracy: 0.6046 - val_loss: 0.6115
Epoch 6/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 89s 216ms/step - accuracy: 0.7272 - loss: 0.5055 - val_accuracy: 0.8819 - val_loss: 0.2643
Epoch 7/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 144s 222ms/step - accuracy: 0.9152 - loss: 0.2220 - val_accuracy: 0.9070 - val_loss: 0.2151
Epoch 8/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 150s 241ms/step - accuracy: 0.9348 - 

Classification A vs. E

In [ ]:
# Load and preprocess data for classes A vs. E
classes_a = ['A']
classes_e = ['E']

X_a, y_a = prepare_data_for_classification(classes_a)
X_e, y_e = prepare_data_for_classification(classes_e)

# Merge and create new labels
X = np.concatenate([X_a, X_e])
y = np.concatenate([np.zeros(len(y_a)), np.ones(len(y_e))])  # 0 for A, 1 for E

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Encode labels
y = y.astype(int)
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the LSTM model
train_and_evaluate_lstm_model(X_train, y_train, X_test, y_test, lstm_units1=128, lstm_units2=64, dense_units=32, dropout_rate=0.3, batch_size=32, epochs=20)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 48s 260ms/step - accuracy: 0.5735 - loss: 0.6459 - val_accuracy: 0.8459 - val_loss: 0.3640
Epoch 2/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 42s 254ms/step - accuracy: 0.8411 - loss: 0.3853 - val_accuracy: 0.8757 - val_loss: 0.3216
Epoch 3/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 40s 245ms/step - accuracy: 0.9086 - loss: 0.2530 - val_accuracy: 0.9275 - val_loss: 0.2106
Epoch 4/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 41s 246ms/step - accuracy: 0.9305 - loss: 0.1905 - val_accuracy: 0.9397 - val_loss: 0.1698
Epoch 5/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 39s 234ms/step - accuracy: 0.9410 - loss: 0.1681 - val_accuracy: 0.9481 - val_loss: 0.1596
Epoch 6/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 44s 250ms/step - accuracy: 0.9421 - loss: 0.1617 - val_accuracy: 0.9474 - val_loss: 0.1628
Epoch 7/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 82s 250ms/step - accuracy: 0.9475 - loss: 0.1500 - val_accuracy: 0.9161 - val_loss: 0.2235
Epoch 8/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 80s 239ms/step - accuracy: 0.9443 - loss: 0

Classification B vs. E

In [ ]:
# Load and preprocess data for classes B vs. E
classes_b = ['B']
classes_e = ['E']

X_b, y_b = prepare_data_for_classification(classes_b)
X_e, y_e = prepare_data_for_classification(classes_e)

# Merge and create new labels
X = np.concatenate([X_b, X_e])
y = np.concatenate([np.zeros(len(y_b)), np.ones(len(y_e))])  # 0 for B, 1 for E

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Encode labels
y = y.astype(int)
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the LSTM model
train_and_evaluate_lstm_model(X_train, y_train, X_test, y_test, lstm_units1=128, lstm_units2=64, dense_units=32, dropout_rate=0.3, batch_size=32, epochs=20)


Classification B vs. D

In [ ]:
# Load and preprocess data for classes B vs. D
classes_b = ['B']
classes_d = ['D']

X_b, y_b = prepare_data_for_classification(classes_b)
X_d, y_d = prepare_data_for_classification(classes_d)

# Merge and create new labels
X = np.concatenate([X_b, X_d])
y = np.concatenate([np.zeros(len(y_b)), np.ones(len(y_d))])  # 0 for B, 1 for D

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Encode labels
y = y.astype(int)
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the LSTM model
train_and_evaluate_lstm_model(X_train, y_train, X_test, y_test, lstm_units1=128, lstm_units2=64, dense_units=32, dropout_rate=0.3, batch_size=32, epochs=20)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 62s 350ms/step - accuracy: 0.6448 - loss: 0.6672 - val_accuracy: 0.5065 - val_loss: 0.6929
Epoch 2/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 63s 237ms/step - accuracy: 0.4883 - loss: 0.6973 - val_accuracy: 0.5149 - val_loss: 0.6924
Epoch 3/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 37s 225ms/step - accuracy: 0.5195 - loss: 0.6944 - val_accuracy: 0.5149 - val_loss: 0.6922
Epoch 4/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 42s 232ms/step - accuracy: 0.4974 - loss: 0.6945 - val_accuracy: 0.4851 - val_loss: 0.6954
Epoch 5/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 40s 228ms/step - accuracy: 0.5360 - loss: 0.6865 - val_accuracy: 0.9634 - val_loss: 0.1467
Epoch 6/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 38s 231ms/step - accuracy: 0.9842 - loss: 0.0733 - val_accuracy: 0.9992 - val_loss: 0.0067
Epoch 7/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 42s 240ms/step - accuracy: 0.9969 - loss: 0.0192 - val_accuracy: 1.0000 - val_loss: 6.0613e-04
Epoch 8/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 38s 224ms/step - accuracy: 0.9982 - los

Classification A vs. C

In [ ]:
# Load and preprocess data for classes A vs. C
classes_a = ['A']
classes_c = ['C']

X_a, y_a = prepare_data_for_classification(classes_a)
X_c, y_c = prepare_data_for_classification(classes_c)

# Merge and create new labels
X = np.concatenate([X_a, X_c])
y = np.concatenate([np.zeros(len(y_a)), np.ones(len(y_c))])  # 0 for A, 1 for C

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Encode labels
y = y.astype(int)
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the LSTM model
train_and_evaluate_lstm_model(X_train, y_train, X_test, y_test, lstm_units1=128, lstm_units2=64, dense_units=32, dropout_rate=0.3, batch_size=32, epochs=20)


Classification C vs. E

In [ ]:
# Load and preprocess data for classes C vs. E
classes_c = ['C']
classes_e = ['E']

X_c, y_c = prepare_data_for_classification(classes_c)
X_e, y_e = prepare_data_for_classification(classes_e)

# Merge and create new labels
X = np.concatenate([X_c, X_e])
y = np.concatenate([np.zeros(len(y_c)), np.ones(len(y_e))])  # 0 for C, 1 for E

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Encode labels
y = y.astype(int)
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the LSTM model
train_and_evaluate_lstm_model(X_train, y_train, X_test, y_test, lstm_units1=128, lstm_units2=64, dense_units=32, dropout_rate=0.3, batch_size=32, epochs=20)
